# Machine Learning Lab: A Simple Neural Network on MNIST with JAX

Our hands-on lab will:

1. Downloads and loads MNIST into NumPy arrays (if it doesn't already exist locally).
2. Builds a simple Multi-Layer Perceptron in JAX.
3. Trains the network on MNIST.
4. Evaluates the performance on test data.
5. Provides a custom inference function for your own handwriting images.

## MNIST Data Loader

We start by downloading the MNIST data set and store it locally.
Our data loader will parse, reshape, normalize them, and return them in NumPy arrays.

In [ ]:
from os.path import isfile
from urllib.request import urlretrieve

base_url = "https://storage.googleapis.com/cvdf-datasets/mnist/"

# File names
files = {
    "train_images": "train-images-idx3-ubyte.gz",
    "train_labels": "train-labels-idx1-ubyte.gz",
    "test_images":  "t10k-images-idx3-ubyte.gz",
    "test_labels":  "t10k-labels-idx1-ubyte.gz",
}

for key, file in files.items():
    if not isfile(file):
        url = base_url + file
        print(f"Downloading {url} to {file}...")
        urlretrieve(url, file)
    else:
        print(f"{file} exists; skip download")

In [ ]:
import gzip
import struct
import array
from jax import numpy as np

# Parsing functions
def parse_labels(file_path):
    with gzip.open(file_path, "rb") as fh:
        _magic, num_data = struct.unpack(">II", fh.read(8))
        # Read the label data as 1-byte unsigned integers
        return np.array(array.array("B", fh.read()), dtype=np.uint8)

def parse_images(file_path):
    with gzip.open(file_path, "rb") as fh:
        _magic, num_data, rows, cols = struct.unpack(">IIII", fh.read(16))
        # Read the image data as 1-byte unsigned integers
        images = np.array(array.array("B", fh.read()), dtype=np.uint8)
        # Reshape to (num_data, 28, 28)
        images = images.reshape(num_data, rows, cols)
        return images

In [ ]:
# Parse raw data

train_images_raw = parse_images(files["train_images"])
train_labels_raw = parse_labels(files["train_labels"])

test_images_raw  = parse_images(files["test_images"])
test_labels_raw  = parse_labels(files["test_labels"])

In [ ]:
# Standardize the images, i.e., flatten and normalize images to [0, 1]
def standardize(images):
    return images.reshape(-1, 28*28).astype(np.float32) / 255

train_images = standardize(train_images_raw)
test_images  = standardize(test_images_raw)

In [ ]:
# One-hot encode labels
def one_hot(labels, num_classes=10):
    return np.eye(num_classes)[labels]

train_labels = one_hot(train_labels_raw, 10).astype(np.float32)
test_labels  = one_hot(test_labels_raw,  10).astype(np.float32)

## Define a Simple Neural Network in JAX

In this subsection, we introduce the core function needed to **initialize** the parameters of a multi-layer network.
Our network will have multiple layers, each characterized by a weight matrix `W` and a bias vector `b`.
We will use random initialization scaled by a small factor to ensure stable starting values for training.

In [ ]:
import numpy.random as npr

def init_params(scale, layer_sizes, rng=npr.RandomState(0)):
    """
    Initialize the parameters (weights and biases) for each layer in the network.

    Parameters
    ----------
    scale : float
        A scaling factor to control the initial range of the weights.
    layer_sizes : list of int
        The sizes of each layer in the network.
        e.g., [784, 1024, 1024, 10] means:
            - Input layer: 784 units
            - Hidden layer 1: 1024 units
            - Hidden layer 2: 1024 units
            - Output layer: 10 units
    rng : numpy.random.RandomState
        Random state for reproducibility.

    Returns
    -------
    params : list of tuples (W, b)
        Each tuple contains (W, b) for a layer.
        - W is a (input_dim, output_dim) array of weights
        - b is a (output_dim,) array of biases
    """
    return [
        (scale * rng.randn(m, n), scale * rng.randn(n))
        for m, n in zip(layer_sizes[:-1], layer_sizes[1:])
    ]

How it works:
* We specify a list of layer sizes: for example, `[784, 1024, 1024, 10]`.
* For each pair of consecutive sizes `(m, n)`, we create a weight matrix W of shape `(m, n)` and a bias vector `b` of shape `(n,)`.
* Multiplying by scale ensures that initial values are not too large, which helps prevent numerical issues early in training.
* We store all `(W, b)` pairs in a list, one pair per layer, to be used throughout training and inference.

By calling `init_params(scale, layer_sizes)`, you obtain an easy-to-manipulate structure that keeps all the parameters needed for your neural network.

In [ ]:
# Define network architecture and hyperparameters

layer_sizes = [784, 1024, 1024, 10]  # 2 hidden layers
param_scale = 0.1

In [ ]:
# Initialize parameters

params = init_params(param_scale, layer_sizes)